In [29]:
import re
import numpy as np

def remove_docstrings(code):
    # 使用正则表达式匹配被 """ 包括的部分，并替换为空字符串
    cleaned_code = re.sub(r'""".*?"""', '', code, flags=re.DOTALL).strip()
    cleaned_code = re.sub(r"'''.*?'''", '', cleaned_code, flags=re.DOTALL).strip()
    cleaned_code = re.sub(r'#.*$', '', cleaned_code, flags=re.MULTILINE).strip()
    return cleaned_code

def cosine_similarity_matrix(nl_features, code_features):
    # 计算每个特征向量的范数
    nl_norms = np.linalg.norm(nl_features, axis=1, keepdims=True)
    code_norms = np.linalg.norm(code_features, axis=1, keepdims=True)
    
    # 计算点积
    dot_product = np.dot(nl_features, code_features.T)
    
    # 计算余弦相似度矩阵
    cosine_similarity = dot_product / (nl_norms * code_norms.T)
    
    return cosine_similarity

In [1]:
import json
input_path = "/home/yiming/cophi/projects/fork/CodeBERT/GraphCodeBERT/codesearch/sorted_label_human_auto.jsonl"
idx_list = []
match_list = []

with open(input_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip().rstrip(',')  # 去除行末的逗号
        json_obj = json.loads(line)
        idx_list.append(json_obj['idx'])
        match_list.append(json_obj['match'])

In [3]:
import json
import os

# 文件路径
json_file_path = '/home/yiming/cophi/training_dynamic/gcb_tokens_temp/Model/Epoch_1/tokenized_code_tokens_train.json'

# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as f:
    code_tokens_data = json.load(f)

# 文件路径
json_file_path = '/home/yiming/cophi/training_dynamic/gcb_tokens_temp/Model/Epoch_1/tokenized_comment_tokens_train.json'


# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as f:
    nl_tokens_data = json.load(f)

In [5]:
for ind in range(len(idx_list)):
    cur_match_list = match_list[ind]
    cur_idx = idx_list[ind]

    teach_code_tokens = code_tokens_data[cur_idx][1:]
    teach_comment_tokens = nl_tokens_data[cur_idx][1:]

    teacher_output = ""
    match_idx = 0
    print(cur_idx)
    # 遍历 match_list
    for match_item in cur_match_list:
        match_idx += 1
        comment_match = match_item[0]
        code_match = match_item[1]
        matched_comment_tokens = []
        for i in range(0, len(comment_match), 2):
            comment_start, comment_end = comment_match[i], comment_match[i+1]
            matched_comment_tokens.extend(teach_comment_tokens[comment_start:comment_end+1])  # 提取代码 tokens
        
        # 处理代码 tokens 的区间，成对提取 [start, end]
        matched_code_tokens = []
        for i in range(0, len(code_match), 2):
            code_start, code_end = code_match[i], code_match[i+1]
            matched_code_tokens.extend(teach_code_tokens[code_start:code_end+1])  # 提取代码 tokens
        
        teacher_output += f"{match_idx}. {matched_comment_tokens}, {matched_code_tokens}\n"
    print(teacher_output)

25
1. ['Check', 'Ġrepository', 'Ġexistence'], ['Ġcheck', '_', 're', 'pos', 'itory', '_', 'existence', 'Ġ(', 'Ġparams', 'Ġ)', 'Ġ:', 'Ġrep', 'od', 'ir', 'Ġ=', 'Ġos', 'Ġ.', 'Ġpath', 'Ġ.', 'Ġjoin', 'Ġ(', 'Ġparams', 'Ġ.', 'Ġout', 'dir', 'Ġ,', 'Ġparams', 'Ġ.', 'Ġname', 'Ġ)', 'Ġif', 'Ġos', 'Ġ.', 'Ġpath', 'Ġ.', 'Ġis', 'dir', 'Ġ(', 'Ġrep', 'od', 'ir', 'Ġ)', 'Ġ:', 'Ġraise']
2. ['Ġrepository', 'Ġ.'], ['ĠConflict', 'Ġ(', "Ġ'", 'Package', 'Ġrepository', 'Ġ"{', '0', '}"', 'Ġhas', 'Ġalready', 'Ġexists', ".'", 'Ġ.', 'Ġformat', 'Ġ(', 'Ġrep', 'od', 'ir', 'Ġ)', 'Ġ)']

32
1. ['Con', 'verts', 'Ġthe'], ['_', 'return', '_', 'type']
2. ['Ġreturned', 'Ġvalue', 'Ġof'], ['Ġ(', 'Ġ*', 'Ġargs', 'Ġ,', 'Ġ**', 'Ġk', 'w', 'args', 'Ġ)', 'Ġreturn', 'Ġf', 'Ġ(']
3. ['Ġwrapped', 'Ġfunction'], ['Ġwraps', 'Ġ(', 'Ġf', 'Ġ)']
4. ['Ġto', 'Ġthe', 'Ġtype', 'Ġof'], ['Ġreturn', '_', 'type', 'Ġ=']
5. ['Ġby', 'Ġa', 'Ġk', 'w', 'arg', 'Ġkey'], ['Ġif', 'Ġk', 'w', 'args', 'Ġ.', 'Ġhas', '_', 'key', 'Ġ(']
6. ['Ġreturn', '_', 'type'], ['retur

In [1]:
import json
input_path = "/home/yiming/cophi/projects/fork/CodeBERT/GraphCodeBERT/codesearch/label_human.jsonl"
idx_list = []
match_list = []

with open(input_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip().rstrip(',')  # 去除行末的逗号
        json_obj = json.loads(line)
        idx_list.append(json_obj['idx'])
        match_list.append(json_obj['match'])

In [4]:
for ind in range(len(idx_list)):
    cur_match_list = match_list[ind]
    cur_idx = idx_list[ind]

    teach_code_tokens = code_tokens_data[cur_idx][1:]
    teach_comment_tokens = nl_tokens_data[cur_idx][1:]

    teacher_output = ""
    match_idx = 0
    print(cur_idx)
    # 遍历 match_list
    for match_item in cur_match_list:
        match_idx += 1
        comment_match = match_item[0]
        code_match = match_item[1]
        matched_comment_tokens = []
        for i in range(0, len(comment_match), 2):
            comment_start, comment_end = comment_match[i], comment_match[i+1]
            matched_comment_tokens.extend(teach_comment_tokens[comment_start:comment_end+1])  # 提取代码 tokens
        
        # 处理代码 tokens 的区间，成对提取 [start, end]
        matched_code_tokens = []
        for i in range(0, len(code_match), 2):
            code_start, code_end = code_match[i], code_match[i+1]
            matched_code_tokens.extend(teach_code_tokens[code_start:code_end+1])  # 提取代码 tokens
        
        teacher_output += f"{match_idx}. {matched_comment_tokens}, {matched_code_tokens}\n"
    print(teacher_output)

546
1. ['Che', 'cks', 'Ġif'], ['Ġvalidate', 'Ġif', 'Ġel', 'if', 'Ġif', 'Ġnot', 'Ġelse', 'Ġelse']
2. ['Ġfor', 'Ġeach', 'Ġedited', 'Ġword'], ['words', 'Ġword', 'Ġfor', 'Ġword', 'Ġin']
3. ['Ġword', '_', 'list'], ['Ġword', '_', 'list', 'Ġword', '_', 'list', 'Ġword', '_', 'list', 'Ġword', '_', 'list', 'Ġword', '_', 'list']
4. ['Ġis', 'Ġa', 'Ġvalid', 'Ġenglish', 'Ġword', 'Ġ.', 'Ġabs'], ['Ġword', 'Ġin', 'ĠW', 'ORD', '_', 'DIS', 'TR', 'IB', 'UTION']
5. ['ĠReturns'], ['Ġreturn', 'Ġreturn', 'Ġreturn', 'Ġraise', 'ĠInput', 'Error']
6. ['Ġset', 'Ġinstance'], ['Ġset']

11024
1. ['S', 'ets'], ['Ġset', 'Ġ=', 'Ġ=', 'Ġ=']
2. ['Ġtime'], ['Ġtime', 'Ġtime', 'time']
3. ['Ġin', 'Ġseconds'], ['Ġ_', 'pb', 'Ġ.', 'Ġsec', 'Ġint', 'Ġ(', 'Ġself', 'Ġ.', 'Ġ_', 'time', 'Ġ_', 'pb', 'Ġ.', 'Ġsec']

19222
1. ['Pl', 'ots'], ['Ġplot', 'Ġensemble', 'Ġ,', 'Ġax', 'Ġ=', 'ĠNone', 'Ġ,', 'Ġj', 'itter', 'Ġ=', 'Ġ0', '.', '1', 'Ġ,', 'Ġscatter', '_', 'kw', 'Ġ=', 'Ġdict', 'Ġ(', 'Ġ)', 'Ġ,', 'Ġline', '_', 'kw', 'Ġ=', 'Ġdict', 'Ġ(', 'Ġ)',

In [4]:
import json
import os

# 文件路径
json_file_path = '/home/yiming/cophi/training_dynamic/gcb_tokens_temp/Model/Epoch_1/tokenized_code_tokens_train.json'

# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as f:
    code_tokens_data = json.load(f)

# 文件路径
json_file_path = '/home/yiming/cophi/training_dynamic/gcb_tokens_temp/Model/Epoch_1/tokenized_comment_tokens_train.json'


# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as f:
    nl_tokens_data = json.load(f)

In [5]:
with open("/home/yiming/cophi/projects/fork/CodeBERT/GraphCodeBERT/codesearch/dataset/python/train.jsonl", "r") as f:
    train_dataset = [json.loads(line) for line in f.readlines()]

In [169]:
import json
input_path = "/home/yiming/cophi/projects/fork/CodeBERT/GraphCodeBERT/codesearch/labelling_sample.jsonl"
idx_list = []
match_list = []

with open(input_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip().rstrip(',')  # 去除行末的逗号
        json_obj = json.loads(line)
        idx_list.append(json_obj['idx'])
        match_list.append(json_obj['match'])

In [170]:
# 预设10种高对比度颜色
high_contrast_colors = [
    "#FF0000", "#00FF00", "#0000FF", "#FF00FF","#00FFFF", 
    "#800000", "#008000", "#FFFF00", "#000080", "#808000"
]

In [363]:
comment_id = code_ind = idx_list[90]
print(comment_id)

246579


In [364]:
indices = [i for i, value in enumerate(idx_list) if value == comment_id]

# 提取match_list中对应索引的值
match_values = [match_list[i] for i in indices]

# 输出结果
print(f"Indices of {comment_id} in idx_list is {indices}")
print("Corresponding values in match_list:", match_values)
# 初始化结果列表
comment_list = []
code_list = []

# 遍历 match_values，处理每个 comment 和 code 的区间
for match in match_values[0]:  # 假设 match_values 的结构是 [[...]]
    comment_intervals, code_intervals = match

    # 展开 comment 的所有索引
    comment_indices = []
    for start, end in zip(comment_intervals[::2], comment_intervals[1::2]):
        comment_indices.extend(range(start, end + 1))
    comment_list.append(comment_indices)

    # 展开 code 的所有索引
    code_indices = []
    for start, end in zip(code_intervals[::2], code_intervals[1::2]):
        code_indices.extend(range(start, end + 1))
    code_list.append(code_indices)

# # 输出结果
# print("Comment indices list:", comment_list)
# print("Code indices list:", code_list)


Indices of 246579 in idx_list is [90]
Corresponding values in match_list: [[[[1, 3, 5, 7], [1, 4, 15, 24, 84, 108]], [[13, 14], [48, 74]]]]


In [365]:
import numpy as np
from collections import deque
from IPython.display import display, HTML

token_list1 = nl_tokens_data[comment_id][1:]
doc_snippet = train_dataset[comment_id]["docstring"]

# 将 token 与对应的贡献值配对，并去掉 token 中的 "Ġ" 符号
tokens = deque([token.replace("Ġ", "") for token in token_list1])

# 为每个 comment_list 的索引组指定颜色
color_map = {}
for i, indices in enumerate(comment_list):
    color = high_contrast_colors[i % len(high_contrast_colors)]  # 循环使用颜色
    for idx in indices:
        color_map[idx] = color

# 生成HTML带颜色调整的输出
html_string = "<pre>"
buffer = ""
current_index = 0  # 追踪 code_snippet 中字符的索引位置

# 遍历原始代码片段的每个字符
for char in doc_snippet:
    if char == "\n":
        # 遇到换行符则添加 <br> 标签并清空缓冲区
        html_string += buffer + "<br>"
        buffer = ""
    elif tokens:
        # 获取当前 token
        token = tokens[0]
        buffer += char

        # 逐字符匹配：检查 token 是否与 buffer 逐字符匹配
        if buffer == token:
            color = color_map.get(current_index, "black")  # 如果索引有颜色则应用，否则默认为黑色
            html_string += f'<span style="color: {color};">{buffer}</span>'
            buffer = ""  # 清空缓冲区
            tokens.popleft()  # 移除已匹配的 token
            current_index += 1
        elif not token.startswith(buffer):
            # 如果缓冲区字符序列和当前 token 不匹配，将缓冲区第一个字符添加到 HTML 并继续逐字符匹配
            color = color_map.get(current_index, "black")
            html_string += f'<span style="color: {color};">{buffer[0]}</span>'
            buffer = buffer[1:]      
        
    else:
        # 如果没有更多 token 需要匹配，默认输出字符
        color = color_map.get(current_index, "black")
        html_string += f'<span style="color: {color};">{char}</span>'

html_string += buffer  # 添加剩余的缓冲区内容
html_string += "</pre>"

# 在Jupyter Notebook中显示带有颜色调整的文本
display(HTML(html_string))

In [366]:
# 假设我们有原始代码片段和分词后的 tokens（包含缩进和换行）
code_snippet = remove_docstrings(train_dataset[code_ind]["code"])
token_list2 = code_tokens_data[code_ind][1:]

# 将 token 中的 "Ġ" 符号去掉
tokens = deque([token.replace("Ġ", "") for token in token_list2])

# 为每个 code_list 的索引组指定颜色
color_map = {}
for i, indices in enumerate(code_list):
    color = high_contrast_colors[i % len(high_contrast_colors)]  # 循环使用颜色
    for idx in indices:
        color_map[idx] = color

# 生成HTML带颜色调整的输出
html_string = "<pre>"
buffer = ""
current_index = 0  # 追踪 code_snippet 中字符的索引位置

# 遍历原始代码片段的每个字符
for char in code_snippet:
    if char == "\n":
        # 遇到换行符则添加 <br> 标签并清空缓冲区
        html_string += buffer + "<br>"
        buffer = ""
    elif tokens:
        # 获取当前 token
        token = tokens[0]
        buffer += char

        # 逐字符匹配：检查 token 是否与 buffer 逐字符匹配
        if buffer == token:
            color = color_map.get(current_index, "black")  # 如果索引有颜色则应用，否则默认为黑色
            html_string += f'<span style="color: {color};">{buffer}</span>'
            buffer = ""  # 清空缓冲区
            tokens.popleft()  # 移除已匹配的 token
            current_index += 1
        elif not token.startswith(buffer):
            # 如果缓冲区字符序列和当前 token 不匹配，将缓冲区第一个字符添加到 HTML 并继续逐字符匹配
            color = color_map.get(current_index, "black")
            html_string += f'<span style="color: {color};">{buffer[0]}</span>'
            buffer = buffer[1:]      
        
    else:
        # 如果没有更多 token 需要匹配，默认输出字符
        color = color_map.get(current_index, "black")
        html_string += f'<span style="color: {color};">{char}</span>'

html_string += buffer  # 添加剩余的缓冲区内容
html_string += "</pre>"

# 在Jupyter Notebook中显示带有颜色调整的文本
display(HTML(html_string))